# Libraries & Constants

In [2]:
__author__ = "Gabriele Greco"
import random
import logging
from collections import namedtuple
from copy import deepcopy
from itertools import accumulate
from operator import xor

# Nim Class

In [3]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [4]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

# Task 1: Fixed rules

In [19]:
def pure_random(state: Nim) -> Nimply: # take a random row and select random elements (always < k)
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    if(state.rows[row] > state.k):
        num_objects = random.randint(1, state.k)
    else:
        num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def my_strategy(state: Nim) -> Nimply:
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] <= state.k): # row elements are less or equal than k, select all the elements and close the row/win
        num_objects = state.rows[row]
    else:
        if(state.rows[row] > (state.k*2)): # if elements are greater than 2k, select k elements
            num_objects = state.k
        else:
            num_objects = state.rows[row] - state.k - 1 # otherwise we want to select a certain amount of elements in order for the opponent to not win
    return Nimply(row, num_objects)

def shortest_row(state: Nim) -> Nimply: # take the shortest row and select random elements if the matches > k otherwise close the row
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] > state.k):
       num_objects = random.randint(1, state.k)
    else:
       num_objects = state.rows[row]
    return Nimply(row, num_objects)

# Nim-sum

In [20]:
# algorithm taken from professor's code
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

# Main

In [23]:
def play(N, turn):
    countwin = 0
    while(N):
        rows = random.randint(5, 13) # number of rows
        k = random.randint(2, 7) # upperbound of selected matches
        if(turn == 0): # who starts first
           player = 1
        else:
           player = 0
        
        nim = Nim(rows, k) #creating the nim
        while(nim):
           if(player == 0):
              #ply = pure_random(nim)
              ply = optimal_strategy(nim)
              #ply = shortest_row(nim)
           else:
              ply = my_strategy(nim)
           nim.nimming(ply)
           player = 1 - player

        if(1 - player == 1): # winner = 1 - player
               countwin += 1  
        N -= 1             

    return countwin

In [24]:
logging.getLogger().setLevel(logging.DEBUG)

for N in [10, 100, 1000]: # game to played as First Player
    countwin = play(N, 0)
    logging.info(f"Game played = {N}: " f"Winrate 1° player = {(countwin/N)*100}% ")
for N in [10, 100, 1000]: # game to played as Second Player
    countwin = play(N, 1)
    logging.info(f"Game played = {N}: " f"Winrate 2° player = {(countwin/N)*100}% ")


INFO:root:Game played = 10: Winrate 1° player = 90.0% 
INFO:root:Game played = 100: Winrate 1° player = 93.0% 
INFO:root:Game played = 1000: Winrate 1° player = 96.1% 
INFO:root:Game played = 10: Winrate 2° player = 100.0% 
INFO:root:Game played = 100: Winrate 2° player = 94.0% 
INFO:root:Game played = 1000: Winrate 2° player = 96.8% 
